In [2]:
import os
import pyspark.sql.functions as funcs
from pyspark.sql import Row
import time

import re

THRESHOLD = 0.001

if not os.path.isfile("test.txt"):
    print("please add: " + os.getcwd())
    FileText= os.path.file("test.txt")


In [3]:

%pylab inline
from pyspark import SparkContext, SparkConf

# transfer df to rdd
dfFile = spark.read.load("C:\\Users\\sgoel\\PycharmProjects\\University\\BigData\\TargilTwo\\test.txt",
                     format="csv", sep='\t', inferSchema="true", header="true")
df = dfFile.select("AnonID", "Query").distinct()
queryData = df.rdd.map(list)



Populating the interactive namespace from numpy and matplotlib


In [4]:
print(queryData.take(2))

[[217, 'mizuno.com'], [1268, 'www.raindanceexpress.com']]


In [5]:
# number of users
usersCount = queryData.groupByKey().count()

In [19]:
queries = queryData.map(lambda line: line[1])
#  support calculation  
querySupport = queries.map(lambda query: (query, 1)) \
    .reduceByKey(lambda k1, k2: k1 + k2) \
    .map(lambda x: (x[0], x[1] / usersCount)) \
    .filter(lambda x: x[1] > THRESHOLD)

remainQueries = querySupport.map(lambda x:x[0]).collect()


In [20]:
print(remainQueries)

['207 ad2d 530', 'www.raindanceexpress.com', 'sstack.com', 'dfdf', 'href a href alt a http www.flickr.com photos 88145967 n00 24368586 in pool-32148876 n00', 'http www.flickr.com photos 88145967 n00 24368586 in pool-32148876 n00', 'ask.com', 'vaniqa.comh', 'www.ngo-quen.org', 'whitepages.com', 'www.victoriacostumiere.com', 'openrangeht.zachsairforce.com', 'ozark horse blankets', 'rapny.com', 'www.ghostrockranch.com', 'yahoo.com', 'broadway.vera.org', 'gallstones', 'asiansexygoddess.com', 'chasebadkids.net', 'myspace.co', 'bestasiancompany.com', 'www.newyorklawyersite.com', 'vietnam', 'vera.org', 'www.buckmountianestates.com', 'united.com', 'www.elaorg', 'www.bonsai.wbff.org', 'frankmellace.com', '-', 'www.tabiecummings.com', 'ucs.ljx.com', 'www.prescriptionfortime.com', 'buddylis', 'westchester.gov', 'gall stones', 'myspace.com', 'space.comhttp', 'lottery', 'googl', 'ameriprise.com', 'susheme', 'rentdirect.com', 'osteen-schatzberg.com', 'idx.techsolsc.com', 'wanttickets.com', 'www.brid

In [26]:
#Remove all quries with lower support then TH
def remainAboveTHQueries(queriesList, remainQueriesList):
    return [s for s in queriesList if s in remainQueriesList]

def createPairs(arr):
        pairs = []
        for q1 in range(len(arr)):
                for q2 in range(q1+1,len(arr)):
                        pairs.append((arr[q1],arr[q2]))
        return pairs
# sort for  (b,a)=>(a,b) 
def sortForUnion(arr):
    if(arr[0] <= arr[1]):
        return arr
    return [arr[1],arr[0]]



In [ ]:
userQueryTuple = queryData.groupByKey().mapValues(list).filter(lambda t: len(t[1]) > 1)  
userQueryTuple = userQueryTuple.map(lambda t: (t[0], remainAboveTHQueries(t[1], remainQueries)))
userQueryTuple.take(2)

In [23]:
quriesTuple = userQueryTuple.map(lambda t: t[1]).flatMap(lambda arr: createPairs(arr))
quriesTuple.take(50)


[('mizuno.com', 'wellsfargo.com'),
 ('mizuno.com', '-'),
 ('mizuno.com', 'united.com'),
 ('mizuno.com', 'yahoo.com'),
 ('mizuno.com', 'asiansexygoddess.com'),
 ('mizuno.com', 'ask.com'),
 ('mizuno.com', 'bestasiancompany.com'),
 ('mizuno.com', 'www.tabiecummings.com'),
 ('mizuno.com', 'weather.com'),
 ('mizuno.com', 'susheme'),
 ('mizuno.com', 'lottery'),
 ('mizuno.com', 'wanttickets.com'),
 ('mizuno.com', "p; .; p;' p; ' ;' ;';"),
 ('mizuno.com', 'www.ngo-quen.org'),
 ('mizuno.com', 'ameriprise.com'),
 ('mizuno.com', 'buddylis'),
 ('mizuno.com', 'vietnam'),
 ('wellsfargo.com', '-'),
 ('wellsfargo.com', 'united.com'),
 ('wellsfargo.com', 'yahoo.com'),
 ('wellsfargo.com', 'asiansexygoddess.com'),
 ('wellsfargo.com', 'ask.com'),
 ('wellsfargo.com', 'bestasiancompany.com'),
 ('wellsfargo.com', 'www.tabiecummings.com'),
 ('wellsfargo.com', 'weather.com'),
 ('wellsfargo.com', 'susheme'),
 ('wellsfargo.com', 'lottery'),
 ('wellsfargo.com', 'wanttickets.com'),
 ('wellsfargo.com', "p; .; p;' p

In [27]:
sortedQueriesTuple = quriesTuple.map(lambda t: sortForUnion(list(t))) \
    .map(lambda arr: (arr[0], arr[1]))

sortedQueriesTupleCount = sortedQueriesTuple \
    .map(lambda t: (t, 1)) \
    .reduceByKey(lambda k1, k2: k1 + k2) \
    .filter(lambda t: t[1] > 1) \
    .map(lambda x: (x[0], x[1] / usersCount))

sortedQueriesTupleCount.take(10)


[]

In [16]:

cartesianTuple = sortedQueriesTupleCount.cartesian(querySupport)
cartesianTuple.take(3)

[((('a', 'b'), 0.6666666666666666), ('a', 1.0)),
 ((('a', 'b'), 0.6666666666666666), ('b', 0.6666666666666666)),
 ((('a', 'b'), 0.6666666666666666), ('c', 0.6666666666666666))]

In [17]:
# calculate confidance (XUY/X by taking all lines that ((x ,y , number of suply(xUy)),(z ,number of suply(z))) when z ==x
rdd_join_left = cartesianTuple.filter(lambda lr: lr[0][0][0] == lr[1][0]) \
                                            .map(lambda lr: (lr[0][0][0],lr[0][0][1],float(lr[0][1]) / lr[1][1]))

print(rdd_join_left.take(5))
# this command take XUY/Y

rdd_join_right = cartesianTuple.filter(lambda lr: lr[0][0][1] == lr[1][0])\
                                             .map(lambda lr: (lr[0][0][1], lr[0][0][0], float(lr[0][1]) / lr[1][1]))
print(rdd_join_right.take(5))

rdd_query_conf = sc.union([rdd_join_left, rdd_join_right])

[('a', 'b', 0.6666666666666666), ('a', 'c', 0.6666666666666666)]
[('b', 'a', 1.0), ('c', 'a', 1.0)]


In [20]:
end = time.time()
elapsed_time = time.strftime("%H:%M:%S", time.gmtime(end - start))
print("elapsed time: %s" % elapsed_time)

elapsed time: 00:00:06


Py4JJavaError: An error occurred while calling o399.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/home/yo/test/Untitled Folder/ex02_all_confs already exists
	at org.apache.hadoop.mapred.FileOutputFormat.checkOutputSpecs(FileOutputFormat.java:131)
	at org.apache.spark.internal.io.HadoopMapRedWriteConfigUtil.assertConf(SparkHadoopWriter.scala:287)
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:71)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1096)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairR